In [ ]:
#Run on Google Colab to use GPU 

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Conv1D, MaxPooling1D, Flatten, Add, ReLU, LSTM, Reshape, Concatenate, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.regularizers import l2

from sklearn.metrics import classification_report, confusion_matrix

import pickle
import os
from contextlib import redirect_stdout

In [ ]:
#import MIT data
df_mitbih_test = pd.read_csv('data/original/mitbih_test.csv', header = None)

X_train = pd.read_csv('data/processed/X_train.csv')
y_train = pd.read_csv('data/processed/y_train.csv')
y_train = y_train['187']

X_train_sm = pd.read_csv('data/processed/X_train_sm.csv')
y_train_sm = pd.read_csv('data/processed/y_train_sm.csv')
y_train_sm = y_train_sm['187']

X_val = pd.read_csv('data/processed/X_val.csv')
y_val = pd.read_csv('data/processed/y_val.csv')
y_val = y_val['187']

X_test = df_mitbih_test.drop(187, axis = 1)
y_test = df_mitbih_test[187]

In [ ]:
#Function to plot and save loss and accuracy over epochs from training history
def plot_training_history(history, save_dir, prefix): 
    hist = history.history
    metrics = [m for m in hist.keys() if not m.startswith('val_')]  

    # Create the output folder if it does not exist
    os.makedirs(save_dir, exist_ok=True)

    for m in metrics:
        plt.figure()
        plt.plot(hist[m], label=f'Train {m}')
        if f'val_{m}' in hist:
            plt.plot(hist[f'val_{m}'], label=f'Val {m}')
        plt.xlabel('Epoch')
        plt.ylabel(m)
        plt.title(f'{m} over epochs')
        plt.legend()
        plt.grid(True)

        # Construct filename with prefix and filepath with directory and filename
        filename = f"{prefix}_{m}.png"
        filepath = os.path.join(save_dir, filename)

        # Save figure
        plt.savefig(filepath, format='png', dpi=300, bbox_inches='tight')
        print(f"Saved: {filepath}")
        plt.show()

In [ ]:
#Used DNNs

#DNN1
dnn1 = Sequential()
dnn1.add(Dense(units=64, activation="relu", input_shape=(187,)))
dnn1.add(Dense(units=32, activation="relu"))
dnn1.add(Dense(units=16, activation="relu"))
dnn1.add(Dense(units=5, activation="softmax"))


#DNN2
dnn2 = Sequential([
    Dense(64, input_shape=(187,), kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),
    
    Dense(32, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),
    
    Dense(16, kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.1),  
    
    Dense(5, activation='softmax')  
])

#DNN3
dnn3 = Sequential([
    Dense(64, input_shape=(187,)),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),
    
    Dense(32),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),
    
    Dense(16),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.1),  
    
    Dense(5, activation='softmax')  
])


#DNN4
dnn4 = Sequential([
    Dense(64, activation='relu', input_shape=(187,), kernel_regularizer=l2(0.001)),
    Dropout(0.2), 
    
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.2),
    
    Dense(16, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.1),
    
    Dense(5, activation='softmax')  
])


In [ ]:
#Model summary
dnn1.summary()

dnn2.summary()

dnn3.summary()

dnn4.summary()

In [ ]:
#reduce learning rate when loss is on plateau
lrredpl = ReduceLROnPlateau(
    monitor='val_loss',  # Metric to monitor
    factor=0.5,          # Factor by which the learning rate will be reduced
    patience=10,         # Number of epochs with no improvement after which learning rate is reduced
    min_lr=1e-7,         # Minimum learning rate
    verbose=1,           # Print message when the learning rate is reduced
    min_delta=0.001      # reduce if improvement < 0.001
)


#learning rate with exponential decay
initial_learning_rate = 5e-4
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96)


#Early stopping
early_stop = EarlyStopping(
    monitor='val_loss',       
    patience=20,               # how many epochs with no improvement before stopping
    restore_best_weights=True, 
    min_delta=0.001            #only stop if improvement < 0.001
)


# Compile the model for lrredpl
#dnn1.compile(
 #   optimizer=Adam(learning_rate=5e-4),
  #  loss='sparse_categorical_crossentropy',
   # metrics=['accuracy', F1Score(average='macro', name='f1_macro')])


#Compile when lr exp decay
dnn3.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])


# Define where and how to save the best model
checkpoint = ModelCheckpoint(
    filepath='../DNN_output/dnn3_sm_expdec5e-4_earlystop20_bs512_epoch_{epoch:02d}_valloss_{val_loss:.4f}.keras',   # file path (can be .keras or .h5)
    monitor='val_loss',        # metric to monitor
    mode='min',                # minimize loss
    save_best_only=False,      # save model for every epoch
    verbose=1                  # print message when model is saved
)

In [ ]:
#Training
history = dnn3.fit(                  # change when using different model architecture
    X_train_sm,
    y_train_sm,
    epochs=500,                      # change when needed
    batch_size=512,                  # change when needed
    validation_data=(X_val, y_val),  # unaltered validation set
    callbacks=[checkpoint, early_stop]          

In [ ]:
#Plot and save loss and accuracy over epochs from history
plot_training_history(history, save_dir="../DNN_output", prefix="dnn3_sm_expdec5e-4_earlystop20_bs512")

In [ ]:
#Save training history
with open("../DNN_output/dnn3_sm_expdec5e-4_earlystop20_bs512_epoch_valloss_.pkl", "wb") as f: #change for model
    pickle.dump(history.history, f)

    
best_model = load_model('../DNN_output/dnn3_sm_expdec5e-4_earlystop20_bs512_epoch_valloss_.keras') #change for model


#prediction of test data
test_pred = best_model.predict(X_test)
y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis=1)


#classification report
print(classification_report(y_test_class, y_pred_class, digits=4))


#confusion matrix
print(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions']))


#save results of metrics
with open("../DNN_output/dnn3_sm_expdec5e-4_earlystop20_bs512_epoch_valloss_.txt", "w") as file: #change for model
    
    file.write("\nModel: DNN1\n")#change for model
        
    file.write("\nData augmentation: Smote\n")
    
    file.write("\nConfusion Matrix on test set:\n")
    file.write(str(pd.crosstab(y_test_class, y_pred_class, colnames=['Predictions'])))
    
    file.write("\n\nClassification Report on test set:\n")
    file.write(classification_report(y_test_class, y_pred_class, digits=4))